In [72]:
import pandas as pd
import numpy as np

from datetime import timedelta
from datetime import datetime


In [2]:
data = pd.read_csv('data/2022.csv')

In [3]:
data = data[data['Actividad_Realizada'] == 'Exploración']
data['Resultado'].value_counts()

Resultado
Negativo      9393
Positivo       891
En proceso       9
Name: count, dtype: int64

In [4]:
data.drop(data[data['Resultado']=='En proceso'].index, inplace=True)

In [ ]:
data_output = data[['Fecha','Latitud','Longitud','Resultado']]
data_output['Fecha'] = data_output['Fecha'].astype('datetime64[s]')

data_output['Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)


In [6]:
df = pd.read_csv('data/ndvi/ndvi_-87.783_21.02725.csv')
df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')
out = df.set_index('date').asfreq('D', method='ffill').reset_index()
out.to_csv('x.csv')

In [99]:
#idx = pd.date_range('2021-09-20', '2021-09-20')

def get_series(folder,field_name,row,names,xmin,xmax):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')  
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    df[names] = (df[names] - xmin )/(xmax-xmin)
    list_f = df[names].values.tolist()  
    

    return list_f

def get_series2(folder,field_name,row,names,xmin,xmax):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Longitud']) + '_' + str(row['Latitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')  
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    df[names] = (df[names] - xmin )/(xmax-xmin)
    list_f = df[names].values.tolist()  
    

    return list_f

def get_series_grop(folder,field_name,row,names,xmin,xmax):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')  
    #df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    df[names] = (df[names] - xmin )/(xmax-xmin)
    list_f = df[names].values.tolist()  
    

    return list_f

#get_series(data_output.iloc[0])['NDVI'].tolist()



In [ ]:
features2 =  ['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']

for feature in features2:
    data_output[feature] = data_output.apply(lambda row: get_series_grop('gee-coordinates/','SOIL',row,feature,0,1), axis=1)


data_output['NDVI'] = data_output.apply(lambda row: get_series('gee-coordinates/','NDVI',row,'NDVI',0,1), axis=1)
data_output['LST_Day_1km'] = data_output.apply(lambda row: get_series('gee-coordinates/','LST_Day_1km',row,'LST_Day_1km',0,1), axis=1)
data_output['NDVI-d'] = data_output.apply(lambda row: get_series2('data/','NDVI',row,'NDVI',0,1), axis=1)
data_output['Lai'] = data_output.apply(lambda row: get_series2('data/','Lai',row,'Lai',0,1), axis=1)


features =  ['prcp','srad','tmax','tmin','vp']

for feature in features:
    data_output[feature] = data_output.apply(lambda row: get_series_grop('gee-coordinates/','DAYMET_V4',row,feature,0,1), axis=1)


dataset = data_output[['Resultado','Lai','NDVI-d','LST_Day_1km','NDVI']+features]

dataset.to_csv('data/2022-dataset.csv', index=False) 

#data_output['MCD15A3H_061_Lai_500m'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_Lai_500m',''), axis=1)
#data_output['MCD15A3H_061_FparExtra_QC'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_FparExtra_QC',''), axis=1)
#data_output['MOD11A1_061_LST_Day_1km'] = data_output.apply(lambda row: get_series(row,'MOD11A1-061','MOD11A1_061_LST_Day_1km',''), axis=1)
#data_output['MOD11A1_061_LST_Night_1km'] = data_output.apply(lambda row: get_series(row,'MOD11A1-061','MOD11A1_061_LST_Night_1km',''), axis=1)


#data_output['MOD13Q1_061__250m_16_days_NDVI'] = data_output.apply(lambda row: get_series(row,'MOD13Q1-061','MOD13Q1_061__250m_16_days_NDVI',''), axis=1)
#data_output['MCD15A3H_061_FparExtra_QC'] = data_output.apply(lambda row: get_series(row,'SPL4SMGP-006','SPL4SMGP_006_Geophysical_Data_sm_rootzone_0',''), axis=1)




In [ ]:
dataset = data_output[['Resultado']+features+features2]
dataset = dataset.loc[:,~dataset.columns.duplicated()].copy()

dataset

In [ ]:

features_all = ['Lai','NDVI-d','LST_Day_1km','NDVI']+['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']+['prcp','srad','tmax','tmin','vp']





#dataset = data_output[['Resultado']+features+features2]

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features_all:
    print(feature)
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: np.mean(row[feature][0:12]),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: np.mean(row[feature][12:24]),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: np.mean(row[feature][24:36]),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: np.mean(row[feature][36:48]),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: np.mean(row[feature][48:60]),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: np.mean(row[feature][60:72]),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: np.mean(row[feature][72:84]),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: np.mean(row[feature][84:96]),axis=1)

feature_dataset.to_csv('data/2022-feature-dataset.csv', index=False) 
feature_dataset.describe()

In [ ]:
data_output['LST_Day_1km'] = data_output.apply(lambda row: get_series('gee-coordinates/','LST_Day_1km',row,'LST_Day_1km',0,1), axis=1)

features = ['LST_Day_1km']

dataset = data_output[['Resultado']+features]

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features:
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: np.mean(row[feature][0:12]),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: np.mean(row[feature][12:24]),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: np.mean(row[feature][24:36]),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: np.mean(row[feature][36:48]),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: np.mean(row[feature][48:60]),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: np.mean(row[feature][60:72]),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: np.mean(row[feature][72:84]),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: np.mean(row[feature][84:96]),axis=1)

feature_dataset.to_csv('data/2022-feature-dataset-LST_Day_1km.csv', index=False) 
feature_dataset.describe()

In [ ]:
features = ['NDVI']

data_output['NDVI'] = data_output.apply(lambda row: get_series('data/','NDVI',row,'NDVI',0,1), axis=1)


dataset = data_output[['Resultado']+features]

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features:
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: np.mean(row[feature][0:12]),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: np.mean(row[feature][12:24]),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: np.mean(row[feature][24:36]),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: np.mean(row[feature][36:48]),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: np.mean(row[feature][48:60]),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: np.mean(row[feature][60:72]),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: np.mean(row[feature][72:84]),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: np.mean(row[feature][84:96]),axis=1)

feature_dataset.to_csv('data/2022-feature-dataset-ndvi-2.csv', index=False) 
feature_dataset.describe()

In [82]:
dataset = pd.read_csv('data/2022-feature-dataset-Lai.csv')
dataset['Resultado'] = dataset.apply(lambda row: 1.0 if row['Resultado'] == 'Positivo' else 0.0,axis=1)
dataset.corr()

,Resultado,Lai_1,Lai_2,Lai_3,Lai_4,Lai_5,Lai_6,Lai_7,Lai_8
Resultado,1.000000,-0.012041,0.016565,0.020469,0.017791,0.025594,0.035995,0.024754,0.019220
Lai_1,-0.012041,1.000000,0.734387,0.653529,0.607440,0.549500,0.473324,0.452186,0.408735
Lai_2,0.016565,0.734387,1.000000,0.732800,0.630539,0.592069,0.540057,0.481053,0.454394
Lai_3,0.020469,0.653529,0.732800,1.000000,0.681076,0.590279,0.565827,0.524375,0.455130
Lai_4,0.017791,0.607440,0.630539,0.681076,1.000000,0.695941,0.596712,0.572501,0.499695
Lai_5,0.025594,0.549500,0.592069,0.590279,0.695941,1.000000,0.690230,0.612876,0.586575
Lai_6,0.035995,0.473324,0.540057,0.565827,0.596712,0.690230,1.000000,0.697582,0.587066
Lai_7,0.024754,0.452186,0.481053,0.524375,0.572501,0.612876,0.697582,1.000000,0.679695
Lai_8,0.019220,0.408735,0.454394,0.455130,0.499695,0.586575,0.587066,0.679695,1.000000


In [117]:
features_selected = ['prcp','srad','tmax','tmin','vp']

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features_selected:
    print(feature)
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: np.mean(row[feature][0:12]),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: np.mean(row[feature][12:24]),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: np.mean(row[feature][24:36]),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: np.mean(row[feature][36:48]),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: np.mean(row[feature][48:60]),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: np.mean(row[feature][60:72]),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: np.mean(row[feature][72:84]),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: np.mean(row[feature][84:96]),axis=1)

feature_dataset.to_csv('data/2022-feature-group1-dataset.csv', index=False) 
feature_dataset.describe()

prcp
srad
tmax
tmin
vp


,prcp_1,prcp_2,prcp_3,prcp_4,prcp_5,prcp_6,prcp_7,prcp_8,srad_1,srad_2,...,tmin_7,tmin_8,vp_1,vp_2,vp_3,vp_4,vp_5,vp_6,vp_7,vp_8
count,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,...,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000,10284.000000
mean,2.123610,2.050912,2.111510,2.120248,2.088025,2.236032,2.192917,2.158350,379.033401,380.284516,...,21.714954,21.770740,2120.034572,2113.404395,2107.153453,2110.693929,2108.387480,2111.351544,2106.173976,2112.492045
std,2.322405,2.442342,2.502046,2.425808,2.374634,2.505966,2.429814,2.300565,58.191554,57.737345,...,1.995698,1.983786,544.450852,552.816569,558.460190,569.668056,574.986925,591.496859,600.402224,600.640968
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,278.922498,278.486665,...,16.850834,16.585000,971.536662,973.412501,953.904160,1001.909999,980.345825,1003.625839,973.412501,984.835007
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,332.420417,332.859169,...,20.314167,20.420417,1747.101034,1749.177915,1732.584780,1704.848333,1687.358551,1679.098956,1660.454788,1657.923136
50%,1.201667,0.948750,0.965833,1.024167,1.026667,1.112917,1.281667,1.311250,368.788335,369.532500,...,21.929167,21.946250,2148.489995,2121.985413,2119.668345,2132.955404,2098.479162,2085.558741,2111.389984,2090.331243
75%,3.601875,3.600833,3.679167,3.803750,3.832917,3.956250,3.781458,3.816875,429.792296,434.154172,...,23.475833,23.501042,2594.915838,2631.463145,2646.220403,2645.582321,2630.869583,2656.211894,2659.718348,2649.509796
max,13.544167,14.271667,14.913333,14.913333,14.015833,14.392500,13.292500,13.056667,525.115827,517.490832,...,24.663333,24.700000,3046.935832,3010.694214,3060.788330,3032.284139,3047.225016,3011.618286,3059.631632,3053.329163
